In [39]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib

from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RationalQuadratic,ConstantKernel
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import minmax_scale

# from sklearn.model_selection import GridSearchCV

In [40]:
# #PATH
path_info = "F:/178/Tugas Akhir/dataset/annotations/"
path_to_fitur = "F:/178/Tugas Akhir/dataset/coba fitur/"
path_to_experiment ="F:/178/Tugas Akhir/dataset/experiment/"
path_to_model = "F:/178/Tugas Akhir/dataset/model/"
path_to_predict = "F:/178/Tugas Akhir/dataset/predict/"

# path_info = "C:/Users/USER/Downloads/Tugas Akhir/dataset/annotations/"
# path_to_fitur = "C:/Users/USER/Downloads/Tugas Akhir/dataset/coba fitur/"
# path_to_experiment ="C:/Users/USER/Downloads/Tugas Akhir/dataset/experiment/"
# path_to_model = "C:/Users/USER/Downloads/Tugas Akhir/dataset/model/"
# path_to_predict = "C:/Users/USER/Downloads/Tugas Akhir/dataset/predict/"

In [41]:
def load(fitur):
    data = pd.read_csv(path_to_experiment + fitur+".csv",header=None)
    ground = pd.read_csv(path_info + "static_annotations.csv")

    arousal_mean = ground['mean_arousal']
    valence_mean = ground['mean_valence']
    
    return data,arousal_mean,valence_mean

def split(data,value):
    train_data, test_data, train_values, test_values = train_test_split(data,value,test_size=0.3,random_state=1000)
    
    return train_data, test_data, train_values, test_values
    
def predict(data,values,test,true,clf):
    trained = clf.fit(data,values)
    pred = clf.predict(test)
    df = pd.DataFrame(data=[pred,true])
    
    return pred,trained,df

def score(pred,true):
    r2 = r2_score(true, pred)
    
    return r2

# def gridsearch(data,true):
#     model = GPR()
# #     1.0 * RationalQuadratic(length_scale=1.0, alpha=0.1)
#     Cs = [1,1e1,1e2,1e3,1e4,1e5]
#     gammas = [1,1e-1,1e-2,1e-3,1e-4,1e-5]
#     parameters = {'C':Cs,'gamma':gammas}

#     clf = GridSearchCV(model, parameters, cv=10, scoring='r2')
#     clf.fit(data,true)

#     return clf.best_params_ , clf.best_score_

def regresion(clf,data,values):
    scores = cross_val_score(clf,data,values,scoring='r2',cv=10)
    
    return scores

def save_model(model,case,types):
    path = path_to_model+types+case+'norm_svr.sav' # save the model
    joblib.dump(model, path)

def save_predict(df1,df2,case):
    path = path_to_predict+case+"_norm_svr.xlsx" # save the model
    arousal = df1.T
    valence = df2.T
    merged = arousal.merge(valence, how='outer', left_index=True, right_index=True)
    merged.to_excel(path,header=None,index=None)

def norm(data):
    scaler.fit(data)
    normalized = scaler.transform(data)
    
    return normalized

# CASE 1

In [42]:
datafitur = "case1" #CASE 1

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

#split arousal
train_data_a, test_data_a, train_values_a, test_values_a = split(data,minmax_scale(arousal_mean.values,feature_range=(-1, 1)))
#split valence
train_data_v, test_data_v, train_values_v, test_values_v = split(data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) 

(744, 26)


GRIDSEARCHCV

In [5]:
# param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
# param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

In [6]:
# print("parameter Arousal:\t",param_a)
# print("parameter Valence:\t",param_v)

INITIAL MODEL

In [43]:
kernels = kernel = ConstantKernel(0.1, (1e-23, 1e5))


svr_case1_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=2, 
                    normalize_y=True, copy_X_train=True, 
                    random_state=1000) #init Arousal

svr_case1_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=2, 
                    normalize_y=True, copy_X_train=True, 
                    random_state=1000) #init Valence

TRAIN

In [45]:
# #Predict Arousal
a_a,train_a,df1_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case1_a)
# #Predict Valence
a_v,train_v,df1_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case1_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [38]:
df1_a = df1_a.round(2)

display(df1_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.09,0.24,0.59,-0.41,-0.41,-0.09,-0.81,0.26,-0.44,-0.09,...,0.49,0.12,0.00,-0.16,-0.06,-0.00,0.32,0.36,-0.23,-0.45
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.64


VALENCE

In [33]:
df1_v = df1_v.round(2)

display(df1_v.head())
print("R2 Score Valence: %0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.16,0.32,0.47,-0.15,-0.07,0.10,-0.31,0.23,-0.08,0.05,...,0.42,0.34,0.01,-0.04,-0.01,0.05,0.16,-0.01,-0.13,-0.44
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence: 0.29


REGRESSION

In [16]:
scores_a = regresion(svr_case1_a,data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #Arousal Regression
scores_v = regresion(svr_case1_v,data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #Arousal Regression

AROUSAL

In [17]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.39104917  0.47461034  0.5163283   0.51305985  0.66234155  0.56973144
  0.51458837  0.59976154  0.64087643  0.52516299]
R2 Score Valence:	0.54 (+/- 0.08)


VALENCE

In [18]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.09028858  0.04210085  0.02166835 -0.16236239  0.34895396  0.08542534
  0.31974883  0.26008757  0.25988991  0.11253767]
R2 Score Valence:	0.14 (+/- 0.15)


In [14]:
# save_predict(df1_a.round(2),df1_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 2

In [19]:
datafitur = "case2" #CASE 2

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #split valence

(744, 34)


GRIDSEARCH

In [16]:
# param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
# param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

In [17]:
# print("parameter Arousal:\t",param_a)
# print("parameter Valence:\t",param_v)

INITIAL MODEL

In [20]:
kernel = RationalQuadratic()

svr_case2_a = GPR(1.0 * kernel) #init Arousal
svr_case2_v = GPR(1.0 * kernel) #init Valence

TRAIN

In [21]:
# #Predict Arousal
a_a,train_a,df2_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case2_a)
# #Predict Valence
a_v,train_v,df2_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case2_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [22]:
df2_a = df2_a.round(2)

display(df2_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.02,0.17,0.48,-0.52,-0.38,-0.16,-0.84,0.27,-0.30,-0.08,...,0.47,0.11,-0.28,-0.29,-0.06,0.03,0.32,0.44,-0.23,-0.32
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.67


VALENCE

In [23]:
df2_v = df2_v.round(2)

display(df2_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.22,0.23,0.40,-0.40,-0.09,-0.01,-0.36,0.32,0.18,0.11,...,0.47,0.50,-0.28,-0.19,-0.05,0.08,0.17,0.12,-0.32,-0.14
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.37


REGRESSION

In [25]:
scores_a = regresion(svr_case2_a,data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #Arousal Regression
scores_v = regresion(svr_case2_v,data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #Arousal Regression

AROUSAL

In [26]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.47787777  0.54028025  0.58891111  0.63616864  0.65247778  0.61616469
  0.63938556  0.65666927  0.59787941  0.58886326]
R2 Score Valence:	0.60 (+/- 0.05)


VALENCE

In [27]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.34236822  0.29545158  0.25635922  0.1281631   0.49426622  0.27268778
  0.46234237  0.34920418  0.32095222  0.17474167]
R2 Score Valence:	0.31 (+/- 0.11)


In [25]:
# save_predict(df2_a.round(2),df2_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 3

In [34]:
datafitur =  "case3" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #split valence

(744, 37)


GRIDSEARCH

In [27]:
# param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
# param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

In [28]:
# print("parameter Arousal:\t",param_a)
# print("parameter Valence:\t",param_v)

INITAL MODEL

In [35]:
kernel = RationalQuadratic()

svr_case3_a = GPR(1.0 * kernel) #init Arousal
svr_case3_v = GPR(1.0 * kernel) #init Valence

TRAIN

In [36]:
# #Predict Arousal
a_a,train_a,df3_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case3_a)
# #Predict Valence
a_v,train_v,df3_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case3_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [37]:
df3_a = df3_a.round(2)

display(df3_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.01,0.16,0.50,-0.53,-0.35,-0.16,-0.84,0.31,-0.33,-0.07,...,0.49,0.11,-0.25,-0.30,-0.11,0.01,0.34,0.44,-0.24,-0.32
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.68


VALENCE

In [38]:
df3_v = df3_v.round(2)

display(df3_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.22,0.22,0.39,-0.36,-0.08,-0.02,-0.36,0.37,0.12,0.12,...,0.48,0.50,-0.28,-0.19,-0.06,0.08,0.21,0.14,-0.32,-0.13
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.39


REGRESSION

In [40]:
scores_a = regresion(svr_case3_a,data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #Arousal Regression
scores_v = regresion(svr_case3_v,data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #Arousal Regression

AROUSAL

In [41]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.50399117  0.52919372  0.55527064  0.63150964  0.6581898   0.6315405
  0.64936832  0.64136589  0.59243061  0.60826999]
R2 Score Valence:	0.60 (+/- 0.05)


VALENCE

In [42]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.34753602  0.30747672  0.25331332  0.12934792  0.51097606  0.27118933
  0.45620889  0.337906    0.3154712   0.17227984]
R2 Score Valence:	0.31 (+/- 0.11)


In [36]:
# save_predict(df3_a.round(2),df3_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 4

In [44]:
datafitur =  "case4" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #split valence

(744, 61)


GRIDSEAERCH

In [38]:
# param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
# param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

In [39]:
# print("parameter Arousal:\t",param_a)
# print("parameter Valence:\t",param_v)

INITIAL MODEL

In [45]:
kernel = RationalQuadratic()

svr_case4_a = GPR(1.0 * kernel) #init Arousal
svr_case4_v = GPR(1.0 * kernel) #init Valence

TRAIN

In [46]:
# #Predict Arousal
a_a,train_a,df4_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case4_a)
# #Predict Valence
a_v,train_v,df4_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case4_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [47]:
df4_a = df4_a.round(2)

display(df4_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.07,0.16,0.53,-0.60,-0.32,-0.24,-0.72,0.30,-0.28,-0.03,...,0.55,0.13,-0.21,-0.33,-0.08,0.05,0.38,0.48,-0.32,-0.50
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.68


VALENCE

In [48]:
df4_v = df4_v.round(2)

display(df4_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.15,0.29,0.55,-0.17,0.00,-0.02,-0.39,0.34,0.10,0.06,...,0.30,0.11,-0.23,-0.25,-0.11,0.03,0.04,0.17,-0.29,-0.04
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.38


REGRESSION

In [50]:
scores_a = regresion(svr_case4_a,data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #Arousal Regression
scores_v = regresion(svr_case4_v,data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #Arousal Regression

AROUSAL

In [51]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.44683069  0.58337355  0.61120376  0.53486387  0.65843424  0.67018963
  0.64321252  0.66147834  0.62621579  0.66084391]
R2 Score Valence:	0.61 (+/- 0.07)


VALENCE

In [52]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.3189443   0.35024166  0.23096405  0.05286824  0.40336337  0.31153572
  0.42609053  0.2952219   0.2768248   0.28358114]
R2 Score Valence:	0.29 (+/- 0.10)


In [47]:
# save_predict(df4_a.round(2),df4_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence